# ESCENARIO PRESENTE (2010)

# PAQUETES

In [2]:
import rasterio
import geopandas as gpd
import numpy as np
import pandas as pd
from rasterio.mask import mask
from shapely.geometry import box
import matplotlib.pyplot as plt

# CARGAR DATOS

In [2]:
#Cargar el grid
Grid_50 = gpd.read_file(r'Data/Grid50/Grid_50Km_Europe_WGS84.shp')
Grid_50

,ET_ID,COUNTRY,geometry
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839..."
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449..."
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029..."
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773..."
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738..."
...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412..."
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708..."
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181..."
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844..."


In [3]:
# Abrir el raster y leer los datos fuera del bucle
src = rasterio.open(r'Data/GLOBIO/2010/Globio4_GBW_LU_2010_10sec.tif')

# Obtener la transformación y el sistema de referencia del raster
raster_transform = src.transform
raster_crs = src.crs


In [4]:
# Reproyectar la malla al sistema de coordenadas del raster si es necesario
if Grid_50.crs != raster_crs:
    Grid_50 = Grid_50.to_crs(raster_crs)

# PROCESAMIENTO DATOS DE HABITAT

Con los datos descargados de GLOBIO para los usos del suelo del año 2010 (periodo coincidente con los datos de temperatura) se procede a extraer la información asociada a cada categoría de hábitat definida por GLOBIO que sea de interés para el modelo de distribución de especies. Para el SDM se ha establecido 4 tipologías de habitas, las cuales están constituidas por distintas categorías de GLOBIO.

- Habitat 1 - Zonas acuáticas
- Habitat 2 - Vegetación Arbórea Perenne
- Habitat 3 - Vegetación Arbórea Caduca
- Habitat 4 - Matorrales y Pastizales


## Habitat 1 - Zonas acuáticas

El habitat 1, corresponderá con ambientes acuaticos, por lo que se incluyen las siguiente tipologias disponible en GLOBIO:
 - 160: Cubierta arbórea inundada con agua dulce o salobre
 - 170: Cubierta arbórea inundada con agua salina
 - 180: Cobertura de arbustos o herbácea inundada con agua dulce/salina/salobre
 - 210: Cuerpos de agua

In [5]:
def extraccion_calculo_area(code):
    # Crear una columna para almacenar el área del código específico por celda
    column_name = f'area_{code}'
    Grid_50[column_name] = 0.0
    
    # Iterar sobre cada celda en la malla
    for idx, cell in Grid_50.iterrows():
        # Crear una máscara para la celda actual
        geom = [cell.geometry]
        out_image, out_transform = mask(src, geom, crop=True, filled=False)
        
        # Extraer el código específico
        code_mask = (out_image == code).astype(np.uint8)
        
        # Calcular el área
        area_code = np.sum(code_mask) * (raster_transform[0] ** 2)  # Área en m²
        
        # Almacenar el resultado en la celda correspondiente
        Grid_50.at[idx, column_name] = area_code

In [6]:
extraccion_calculo_area(160)
extraccion_calculo_area(170)
extraccion_calculo_area(180)
extraccion_calculo_area(210)

In [7]:
Grid_50

,ET_ID,COUNTRY,geometry,area_160,area_170,area_180,area_210
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.000000,0.0,0.000000,0.087106
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.000000,0.0,0.000000,0.224745
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.000000,0.0,0.000000,0.207083
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.000000,0.0,0.000054,0.113789
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.000000,0.0,0.000000,0.123387
...,...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.000000,0.0,0.004437,0.441667
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.000000,0.0,0.012261,0.180910
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.000054,0.0,0.006613,0.129668
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.000000,0.0,0.003565,0.214846


In [8]:
# Seleccionar las columnas
area_columns = [col for col in Grid_50.columns if col.startswith('area_')]
# sumatorio de las distintas columnas con el area de cada tipología
Grid_50['Area_H_1'] = Grid_50[area_columns].sum(axis=1)

In [9]:
Grid_50.head()

,ET_ID,COUNTRY,geometry,area_160,area_170,area_180,area_210,Area_H_1
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.0,0.0,0.000000,0.087106,0.087106
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.0,0.0,0.000000,0.224745,0.224745
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.0,0.0,0.000000,0.207083,0.207083
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.0,0.0,0.000054,0.113789,0.113843
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.0,0.0,0.000000,0.123387,0.123387


In [10]:
# Guardar el resultado en un nuevo shapefile
# Grid_50.to_file('Data/GLOBIO/2010/Grids_Habitats/Grid_H_1.shp')

In [11]:
Grid_50.columns

Index(['ET_ID', 'COUNTRY', 'geometry', 'area_160', 'area_170', 'area_180',
       'area_210', 'Area_H_1'],
      dtype='object')

In [12]:
columns_select = ['ET_ID', 'COUNTRY', 'geometry',
                  'Area_H_1'
                  ]

Grid_Habitats = Grid_50[columns_select]
Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224745
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113843
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123387
...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.446103
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.193171
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136335
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218410


## Habitat 2 - Vegetación Arbórea Perenne

El habitat 2, corresponde con vegetación arbórea perenne, por lo que se incluyen las siguiente tipologias disponible en GLOBIO:

 - 50: Cubierta arbórea de hoja ancha perenne cerrada a abierta (>15%)
 - 70: Cubierta arbórea de hoja acicular perenne cerrada a abierta (>15%)
 - 71: Cubierta arbórea de hoja acicular perenne cerrada (>40%)
 - 72: Cubierta arbórea de hoja acicular perenne abierta (15-40%)

In [13]:
#Cargar el grid nuevamente
Grid_50 = gpd.read_file(r'Grid50/Grid_50Km_Europe_WGS84.shp')
# Reproyectar la malla al sistema de coordenadas del raster si es necesario
if Grid_50.crs != raster_crs:
    Grid_50 = Grid_50.to_crs(raster_crs)

In [14]:
extraccion_calculo_area(50)
extraccion_calculo_area(70)
extraccion_calculo_area(71)
extraccion_calculo_area(72)

In [15]:

area_columns = [col for col in Grid_50.columns if col.startswith('area_')]

Grid_50['Area_H_2'] = Grid_50[area_columns].sum(axis=1)

In [16]:
Grid_50.head()

,ET_ID,COUNTRY,geometry,area_50,area_70,area_71,area_72,Area_H_2
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.0,0.000000,0.0,0.0,0.000000
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.0,0.000000,0.0,0.0,0.000000
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.0,0.000000,0.0,0.0,0.000000
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.0,0.004977,0.0,0.0,0.004977
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.0,0.000378,0.0,0.0,0.000378


In [17]:
# Guardar el resultado en un nuevo shapefile
#Grid_50.to_file('Data/GLOBIO/2010/Grid_H_2.shp')

In [18]:
# Columnas de interes
Grid_50_s = Grid_50[['ET_ID', 'Area_H_2']]

In [19]:
#  Unir los DataFrames usando el campo ET_ID como clave
Grid_Habitats = pd.merge(Grid_Habitats, Grid_50_s, on='ET_ID', how='left')
Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000000
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224745,0.000000
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000000
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113843,0.004977
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123387,0.000378
...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.446103,0.003866
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.193171,0.018920
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136335,0.008858
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218410,0.003511


## Habitat 3 - Vegetación Arbórea Caduca

El habitat 3, corresponde con vegetación arbórea caduca, por lo que se incluyen las siguiente tipologias disponible en GLOBIO:

 - 60: Cubierta arbórea de hoja ancha caduca cerrada a abierta (>15%)
 - 61: Cubierta arbórea de hoja ancha caduca cerrada (>40%)
 - 62: Cubierta arbórea de hoja ancha caduca cerrada (>40%)
 - 80: Cubierta arbórea de hoja acicular caduca cerrada a abierta (>15%)
 - 81: Cubierta arbórea de hoja acicular caduca cerrada (>40%)
 - 82: Cubierta arbórea de hoja acicular caduca abierta (15-40%)

In [20]:
#Cargar el grid nuevamente
Grid_50 = gpd.read_file(r'Grid50/Grid_50Km_Europe_WGS84.shp')


if Grid_50.crs != raster_crs:
    Grid_50 = Grid_50.to_crs(raster_crs)

extraccion_calculo_area(60)
extraccion_calculo_area(61)
extraccion_calculo_area(62)
extraccion_calculo_area(80)
extraccion_calculo_area(81)
extraccion_calculo_area(82)

area_columns = [col for col in Grid_50.columns if col.startswith('area_')]

Grid_50['Area_H_3'] = Grid_50[area_columns].sum(axis=1)

Grid_50.head()

,ET_ID,COUNTRY,geometry,area_60,area_61,area_62,area_80,area_81,area_82,Area_H_3
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.000093,0.0,0.0,0.0,0.0,0.0,0.000093
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.000000,0.0,0.0,0.0,0.0,0.0,0.000000


In [21]:
# Guardar el resultado en un nuevo shapefile
#Grid_50.to_file('Data/GLOBIO/2010/Grid_H_3.shp')

In [22]:
# Columnas de interes
Grid_50_s = Grid_50[['ET_ID', 'Area_H_3']]

#  Unir los DataFrames usando el campo ET_ID como clave
Grid_Habitats = pd.merge(Grid_Habitats, Grid_50_s, on='ET_ID', how='left')
Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2,Area_H_3
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000000,0.000000
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224745,0.000000,0.000000
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000000,0.000000
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113843,0.004977,0.000093
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123387,0.000378,0.000000
...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.446103,0.003866,0.005046
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.193171,0.018920,0.017955
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136335,0.008858,0.013248
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218410,0.003511,0.003403


## Habitat 4 - Matorrales y Pastizales

El habitat 4, corresponde con matorrales y pastizales, por lo que se incluyen las siguiente tipologias disponible en GLOBIO:

 - 10: Tierras de cultivo
 - 13: Mosaico de tierras de cultivo (>50%) / vegetación natural (cubierta herbácea arbustiva) (<50%)
 - 14: Vegetación natural en mosaico (cubierta herbácea arbustiva)
 - 120: Matorrales
 - 121: Matorrales perennes
 - 122: Matorrales caducifolios
 - 130: Pastizales
 - 150: Vegetación dispersa (árboles, arbustos, cobertura herbácea) (<15%)
 - 152: Arbustos dispersos (<15%)
 - 153: Cobertura herbácea dispersa (<15%)
 - 110: Mosaico de cobertura herbácea (>50%) / árboles y arbustos (<50%)

In [23]:
Grid_50 = gpd.read_file(r'Grid50/Grid_50Km_Europe_WGS84.shp')

if Grid_50.crs != raster_crs:
    Grid_50 = Grid_50.to_crs(raster_crs)

extraccion_calculo_area(10)
extraccion_calculo_area(13)
extraccion_calculo_area(14)
extraccion_calculo_area(120)
extraccion_calculo_area(121)
extraccion_calculo_area(122)
extraccion_calculo_area(130)
extraccion_calculo_area(150)
extraccion_calculo_area(152)
extraccion_calculo_area(153)
extraccion_calculo_area(110)

area_columns = [col for col in Grid_50.columns if col.startswith('area_')]

Grid_50['Area_H_4'] = Grid_50[area_columns].sum(axis=1)

Grid_50.head()

,ET_ID,COUNTRY,geometry,area_10,area_13,area_14,area_120,area_121,area_122,area_130,area_150,area_152,area_153,area_110,Area_H_4
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.083796,0.005972,0.003295,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.093063
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.010069,0.002755,0.000571,0.0001,0.0,0.000008,0.0,0.000347,0.0,0.0,0.0,0.013850
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.009568,0.005957,0.009769,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.025293
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.038958,0.000525,0.001736,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.041219
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.046821,0.001559,0.002554,0.0000,0.0,0.000000,0.0,0.000015,0.0,0.0,0.0,0.050949


In [24]:
# Guardar el resultado en un nuevo shapefile
#Grid_50.to_file('Data/GLOBIO/2010/Grid_H_4.shp')

In [25]:
# Columnas de interes
Grid_50_s = Grid_50[['ET_ID', 'Area_H_4']]

#  Unir los DataFrames usando el campo ET_ID como clave
Grid_Habitats = pd.merge(Grid_Habitats, Grid_50_s, on='ET_ID', how='left')
Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2,Area_H_3,Area_H_4
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000000,0.000000,0.093063
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224745,0.000000,0.000000,0.013850
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000000,0.000000,0.025293
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113843,0.004977,0.000093,0.041219
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123387,0.000378,0.000000,0.050949
...,...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.446103,0.003866,0.005046,0.116489
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.193171,0.018920,0.017955,0.273264
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136335,0.008858,0.013248,0.275471
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218410,0.003511,0.003403,0.234113


In [26]:
# Guardar el resultado en un nuevo shapefile
#Grid_Habitats.to_file('Data/GLOBIO/2010/Grid_Habitats.shp')

In [27]:
# Cerrar el dataset raster
src.close()

# % DE ÁREA DISPONIBLE POR CUADRÍCULA

In [28]:
# Calculo del area para cada cuadrícula en km²
Grid_Habitats['Area_grid'] = Grid_Habitats.geometry.area #en m2
Grid_Habitats

C:\Users\molic\AppData\Local\Temp\ipykernel_22232\850118656.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  Grid_Habitats['Area_grid'] = Grid_Habitats.geometry.area #en m2


,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2,Area_H_3,Area_H_4,Area_grid
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000000,0.000000,0.093063,0.246970
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224745,0.000000,0.000000,0.013850,0.249205
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000000,0.000000,0.025293,0.248088
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113843,0.004977,0.000093,0.041219,0.247813
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123387,0.000378,0.000000,0.050949,0.247566
...,...,...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.446103,0.003866,0.005046,0.116489,0.612105
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.193171,0.018920,0.017955,0.273264,0.608670
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136335,0.008858,0.013248,0.275471,0.605030
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218410,0.003511,0.003403,0.234113,0.601196


In [29]:
# Calculo del porcentaje de habitat
Grid_Habitats['%_H_1'] = (Grid_Habitats['Area_H_1']*100)/Grid_Habitats['Area_grid']
Grid_Habitats['%_H_2'] = (Grid_Habitats['Area_H_2']*100)/Grid_Habitats['Area_grid']
Grid_Habitats['%_H_3'] = (Grid_Habitats['Area_H_3']*100)/Grid_Habitats['Area_grid']
Grid_Habitats['%_H_4'] = (Grid_Habitats['Area_H_4']*100)/Grid_Habitats['Area_grid']

Grid_Habitats

,ET_ID,COUNTRY,geometry,Area_H_1,Area_H_2,Area_H_3,Area_H_4,Area_grid,%_H_1,%_H_2,%_H_3,%_H_4
0,249,Spain,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",0.087106,0.000000,0.000000,0.093063,0.246970,35.270045,0.000000,0.000000,37.681992
1,280,Malta,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",0.224745,0.000000,0.000000,0.013850,0.249205,90.184792,0.000000,0.000000,5.557788
2,363,Spain,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",0.207083,0.000000,0.000000,0.025293,0.248088,83.471585,0.000000,0.000000,10.195240
3,412,Greece,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",0.113843,0.004977,0.000093,0.041219,0.247813,45.938994,2.008313,0.037364,16.633191
4,413,Greece,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",0.123387,0.000378,0.000000,0.050949,0.247566,49.840265,0.152722,0.000000,20.580031
...,...,...,...,...,...,...,...,...,...,...,...,...
3101,9369,Norway,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",0.446103,0.003866,0.005046,0.116489,0.612105,72.880249,0.631549,0.824417,19.030928
3102,9370,Norway,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.193171,0.018920,0.017955,0.273264,0.608670,31.736606,3.108375,2.949913,44.895223
3103,9371,Norway,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",0.136335,0.008858,0.013248,0.275471,0.605030,22.533577,1.464064,2.189719,45.530093
3104,9372,Norway,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.218410,0.003511,0.003403,0.234113,0.601196,36.329307,0.583969,0.566001,38.941126


In [30]:
Grid_Habitats_Variables = Grid_Habitats[['ET_ID', 'geometry', '%_H_1', '%_H_2', '%_H_3', '%_H_4']]
Grid_Habitats_Variables

,ET_ID,geometry,%_H_1,%_H_2,%_H_3,%_H_4
0,249,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",35.270045,0.000000,0.000000,37.681992
1,280,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",90.184792,0.000000,0.000000,5.557788
2,363,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",83.471585,0.000000,0.000000,10.195240
3,412,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",45.938994,2.008313,0.037364,16.633191
4,413,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",49.840265,0.152722,0.000000,20.580031
...,...,...,...,...,...,...
3101,9369,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",72.880249,0.631549,0.824417,19.030928
3102,9370,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",31.736606,3.108375,2.949913,44.895223
3103,9371,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",22.533577,1.464064,2.189719,45.530093
3104,9372,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",36.329307,0.583969,0.566001,38.941126


In [ ]:
# Guardar el resultado en un nuevo shapefile
#Grid_Habitats_Variables.to_file('Data/GLOBIO/2010/Grid_Habitats_Variables.shp')

# UNION CON VARIABLES CLIMÁTICAS

In [36]:
#Cargar el grid
#Grid_Habitats_Variables = gpd.read_file(r'Data/GLOBIO/2010/Grid_Habitats_Variables.shp')
Grid_Habitats_Variables

,ET_ID,%_H_1,%_H_2,%_H_3,%_H_4,geometry
0,249,35.270045,0.000000,0.000000,37.681992,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839..."
1,280,90.184792,0.000000,0.000000,5.557788,"POLYGON ((14.68659 35.53849, 14.14122 35.56449..."
2,363,83.471585,0.000000,0.000000,10.195240,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029..."
3,412,45.938994,2.008313,0.037364,16.633191,"POLYGON ((23.95653 35.05738, 23.41935 35.13773..."
4,413,49.840265,0.152722,0.000000,20.580031,"POLYGON ((24.49271 34.97391, 23.95653 35.05738..."
...,...,...,...,...,...,...
3101,9369,72.880249,0.631549,0.824417,19.030928,"POLYGON ((26.92771 70.58708, 25.63446 70.69412..."
3102,9370,31.736606,3.108375,2.949913,44.895223,"POLYGON ((28.20775 70.47181, 26.92771 70.58708..."
3103,9371,22.533577,1.464064,2.189719,45.530093,"POLYGON ((29.47384 70.34844, 28.20775 70.47181..."
3104,9372,36.329307,0.583969,0.566001,38.941126,"POLYGON ((30.72534 70.21713, 29.47384 70.34844..."


In [12]:
import glob
import os

directorio = 'Data/CHELSA/Estadistica_Zona' 
archivos_shp = glob.glob(directorio + '/Bio_*.shp') 
archivos_shp

['C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_01.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_02.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_03.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_04.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_05.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_06.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_07.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_08.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_09.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_10.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_11.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_12.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa_bio/Estadistica_Zona\\Bio_13.shp',
 'C:/Users/molic/Desktop/TFM_V1/Chelsa

In [16]:
# Cargar cada archivo shapefile y asignarlo a una variable con el nombre del archivo
for shp in archivos_shp:
    # Obtener el nombre base del archivo sin la extensión
    file_name = os.path.splitext(os.path.basename(shp))[0]
    
    # Cargar el shapefile en un GeoDataFrame
    gdf = gpd.read_file(shp)
    
    # Asignar el GeoDataFrame a una variable con el nombre del archivo
    globals()[file_name] = gdf

In [17]:
print(Bio_14)

       ET_ID  Bio_14_mea                                           geometry
0      249.0      1.8524  POLYGON ((-2.18083 34.85145, -2.71695 34.77839...
1      280.0      0.2310  POLYGON ((14.68659 35.53849, 14.14122 35.56449...
2      363.0      1.0697  POLYGON ((-2.79499 35.22702, -3.33331 35.15029...
3      412.0      0.9438  POLYGON ((23.95653 35.05738, 23.41935 35.13773...
4      413.0      1.3217  POLYGON ((24.49271 34.97391, 23.95653 35.05738...
...      ...         ...                                                ...
3101  9369.0     47.8207  POLYGON ((26.92771 70.58708, 25.63446 70.69412...
3102  9370.0     56.8986  POLYGON ((28.20775 70.47181, 26.92771 70.58708...
3103  9371.0     54.4348  POLYGON ((29.47384 70.34844, 28.20775 70.47181...
3104  9372.0     50.6536  POLYGON ((30.72534 70.21713, 29.47384 70.34844...
3105  9485.0     45.2222  POLYGON ((28.60702 70.90647, 27.30206 71.02446...

[3106 rows x 3 columns]


In [18]:
# Guardar el resultado en un nuevo shapefile
Bio_01 = Bio_01[['ET_ID', 'Bio_01_mea']]
Bio_02 = Bio_02[['ET_ID', 'Bio_02_mea']]
Bio_03 = Bio_03[['ET_ID', 'Bio_03_mea']]
Bio_04 = Bio_04[['ET_ID', 'Bio_04_mea']]
Bio_05 = Bio_05[['ET_ID', 'Bio_05_mea']]
Bio_06 = Bio_06[['ET_ID', 'Bio_06_mea']]
Bio_07 = Bio_07[['ET_ID', 'Bio_07_mea']]
Bio_08 = Bio_08[['ET_ID', 'Bio_08_mea']]
Bio_09 = Bio_09[['ET_ID', 'Bio_09_mea']]
Bio_10 = Bio_10[['ET_ID', 'Bio_10_mea']]
Bio_11 = Bio_11[['ET_ID', 'Bio_11_mea']]
Bio_12 = Bio_12[['ET_ID', 'Bio_12_mea']]
Bio_13 = Bio_13[['ET_ID', 'Bio_13_mea']]
Bio_14 = Bio_14[['ET_ID', 'Bio_14_mea']]
Bio_15 = Bio_15[['ET_ID', 'Bio_15_mea']]
Bio_16 = Bio_16[['ET_ID', 'Bio_16_mea']]
Bio_17 = Bio_17[['ET_ID', 'Bio_17_mea']]
Bio_18 = Bio_18[['ET_ID', 'Bio_18_mea']]
Bio_19 = Bio_19[['ET_ID', 'Bio_19_mea']]



In [21]:
df_names = ['Bio_01', 'Bio_02', 'Bio_03', 'Bio_04', 'Bio_05', 'Bio_06', 'Bio_07', 'Bio_08', 'Bio_09', 'Bio_10', 'Bio_11', 'Bio_12', 'Bio_13', 'Bio_14', 'Bio_15', 'Bio_16', 'Bio_17', 'Bio_18', 'Bio_19']

In [37]:
for df_name in df_names:
    df = globals()[df_name] 
    Grid_Habitats_Variables = pd.merge(Grid_Habitats_Variables, df, on='ET_ID', how='left')

Grid_Habitats_Variables

,ET_ID,%_H_1,%_H_2,%_H_3,%_H_4,geometry,Bio_01_mea,Bio_02_mea,Bio_03_mea,Bio_04_mea,...,Bio_10_mea,Bio_11_mea,Bio_12_mea,Bio_13_mea,Bio_14_mea,Bio_15_mea,Bio_16_mea,Bio_17_mea,Bio_18_mea,Bio_19_mea
0,249,35.270045,0.000000,0.000000,37.681992,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",18.254976,8.495586,0.368012,462.715716,...,24.509797,12.678282,345.210093,47.583666,1.8524,57.250155,135.057773,13.583357,22.131150,126.752235
1,249,35.270045,0.000000,0.000000,37.681992,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",18.254976,8.495586,0.368012,462.715716,...,24.509797,12.678282,345.210093,47.583666,1.8524,57.250155,135.057773,13.583357,22.131150,126.752235
2,280,90.184792,0.000000,0.000000,5.557788,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",19.165826,1.528587,0.110094,436.010840,...,25.079757,13.986250,510.006213,98.965952,0.2310,83.750042,278.948983,7.215380,52.409418,173.755754
3,363,83.471585,0.000000,0.000000,10.195240,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",18.462150,3.756775,0.238924,374.399104,...,23.658651,14.126988,379.820661,55.684323,1.0697,62.789642,159.041741,12.041937,20.850420,152.538774
4,363,83.471585,0.000000,0.000000,10.195240,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",18.462150,3.756775,0.238924,374.399104,...,23.658651,14.126988,379.820661,55.684323,1.0697,62.789642,159.041741,12.041937,20.850420,152.538774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25703,9372,36.329307,0.583969,0.566001,38.941126,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.627255,3.821428,0.183020,591.614208,...,8.576487,-6.381050,867.108894,100.286265,50.6536,21.261742,269.148897,164.141273,259.901421,205.447719
25704,9485,85.282380,0.424124,0.096730,8.570526,"POLYGON ((28.60702 70.90647, 27.30206 71.02446...",2.420992,2.626016,0.170077,442.528004,...,8.532202,-2.675670,820.016682,98.181465,45.2222,23.091894,255.333051,147.027669,186.234926,233.672052
25705,9485,85.282380,0.424124,0.096730,8.570526,"POLYGON ((28.60702 70.90647, 27.30206 71.02446...",2.420992,2.626016,0.170077,442.528004,...,8.532202,-2.675670,820.016682,98.181465,45.2222,23.091894,255.333051,147.027669,186.234926,233.672052
25706,9485,85.282380,0.424124,0.096730,8.570526,"POLYGON ((28.60702 70.90647, 27.30206 71.02446...",2.420992,2.626016,0.170077,442.528004,...,8.532202,-2.675670,820.016682,98.181465,45.2222,23.091894,255.333051,147.027669,186.234926,233.672052


In [28]:
# Redondear todas las columnas float64 a 4 decimales
Grid_Habitats_Variables = Grid_Habitats_Variables.round(4)

,ET_ID,%_H_1,%_H_2,%_H_3,%_H_4,geometry,Bio_01_mea,Bio_02_mea,Bio_03_mea,Bio_04_mea,...,Bio_10_mea,Bio_11_mea,Bio_12_mea,Bio_13_mea,Bio_14_mea,Bio_15_mea,Bio_16_mea,Bio_17_mea,Bio_18_mea,Bio_19_mea
0,249,35.2700,0.0,0.0,37.6820,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",18.2550,8.4956,0.3680,462.7157,...,24.5098,12.6783,345.2101,47.5837,1.8524,57.2502,135.0578,13.5834,22.1311,126.7522
1,249,35.2700,0.0,0.0,37.6820,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",18.2550,8.4956,0.3680,462.7157,...,24.5098,12.6783,345.2101,47.5837,1.8524,57.2502,135.0578,13.5834,22.1311,126.7522
2,280,90.1848,0.0,0.0,5.5578,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",19.1658,1.5286,0.1101,436.0108,...,25.0798,13.9862,510.0062,98.9660,0.2310,83.7500,278.9490,7.2154,52.4094,173.7558


In [38]:
Grid_Habitats_Variables = Grid_Habitats_Variables.drop_duplicates(subset=['ET_ID'])
Grid_Habitats_Variables

,ET_ID,%_H_1,%_H_2,%_H_3,%_H_4,geometry,Bio_01_mea,Bio_02_mea,Bio_03_mea,Bio_04_mea,...,Bio_10_mea,Bio_11_mea,Bio_12_mea,Bio_13_mea,Bio_14_mea,Bio_15_mea,Bio_16_mea,Bio_17_mea,Bio_18_mea,Bio_19_mea
0,249,35.270045,0.000000,0.000000,37.681992,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",18.254976,8.495586,0.368012,462.715716,...,24.509797,12.678282,345.210093,47.583666,1.8524,57.250155,135.057773,13.583357,22.131150,126.752235
2,280,90.184792,0.000000,0.000000,5.557788,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",19.165826,1.528587,0.110094,436.010840,...,25.079757,13.986250,510.006213,98.965952,0.2310,83.750042,278.948983,7.215380,52.409418,173.755754
3,363,83.471585,0.000000,0.000000,10.195240,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",18.462150,3.756775,0.238924,374.399104,...,23.658651,14.126988,379.820661,55.684323,1.0697,62.789642,159.041741,12.041937,20.850420,152.538774
5,412,45.938994,2.008313,0.037364,16.633191,"POLYGON ((23.95653 35.05738, 23.41935 35.13773...",17.727971,4.337668,0.225087,507.017180,...,24.354709,11.601990,802.802130,166.771665,0.9438,93.524103,465.878952,5.476485,17.501850,370.952439
7,413,49.840265,0.152722,0.000000,20.580031,"POLYGON ((24.49271 34.97391, 23.95653 35.05738...",17.632936,3.921218,0.207843,509.289924,...,24.264651,11.499705,734.030732,147.560539,1.3217,90.566040,412.471176,5.717345,15.655768,337.289896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25678,9369,72.880249,0.631549,0.824417,19.030928,"POLYGON ((26.92771 70.58708, 25.63446 70.69412...",1.797804,3.621791,0.179989,568.053206,...,9.498814,-4.828538,883.689309,104.235966,47.8207,23.038690,277.268834,156.062921,208.810237,253.850528
25685,9370,31.736606,3.108375,2.949913,44.895223,"POLYGON ((28.20775 70.47181, 26.92771 70.58708...",0.008627,4.658969,0.187124,700.483670,...,9.401169,-8.395216,1006.391733,117.775208,56.8986,21.223914,309.381800,183.582187,266.479314,259.674011
25692,9371,22.533577,1.464064,2.189719,45.530093,"POLYGON ((29.47384 70.34844, 28.20775 70.47181...",-0.399243,4.666984,0.187774,700.773789,...,8.995605,-8.801159,961.856564,113.000975,54.4348,21.979814,298.051595,175.460374,274.657953,237.605405
25699,9372,36.329307,0.583969,0.566001,38.941126,"POLYGON ((30.72534 70.21713, 29.47384 70.34844...",0.627255,3.821428,0.183020,591.614208,...,8.576487,-6.381050,867.108894,100.286265,50.6536,21.261742,269.148897,164.141273,259.901421,205.447719


In [39]:
# Redondear todas las columnas float64 a 4 decimales
Grid_Habitats_Variables = Grid_Habitats_Variables.round(4)
Grid_Habitats_Variables.head(3)

,ET_ID,%_H_1,%_H_2,%_H_3,%_H_4,geometry,Bio_01_mea,Bio_02_mea,Bio_03_mea,Bio_04_mea,...,Bio_10_mea,Bio_11_mea,Bio_12_mea,Bio_13_mea,Bio_14_mea,Bio_15_mea,Bio_16_mea,Bio_17_mea,Bio_18_mea,Bio_19_mea
0,249,35.2700,0.0,0.0,37.6820,"POLYGON ((-2.18083 34.85145, -2.71695 34.77839...",18.2550,8.4956,0.3680,462.7157,...,24.5098,12.6783,345.2101,47.5837,1.8524,57.2502,135.0578,13.5834,22.1311,126.7522
2,280,90.1848,0.0,0.0,5.5578,"POLYGON ((14.68659 35.53849, 14.14122 35.56449...",19.1658,1.5286,0.1101,436.0108,...,25.0798,13.9862,510.0062,98.9660,0.2310,83.7500,278.9490,7.2154,52.4094,173.7558
3,363,83.4716,0.0,0.0,10.1952,"POLYGON ((-2.79499 35.22702, -3.33331 35.15029...",18.4622,3.7568,0.2389,374.3991,...,23.6587,14.1270,379.8207,55.6843,1.0697,62.7896,159.0417,12.0419,20.8504,152.5388


In [40]:
# Exportar el resultado
Grid_Habitats_Variables.to_csv('Data/Variables/Grid_Variables.csv', index=False, decimal='.')
Grid_Habitats_Variables.to_file('Data/Variables/Grid_Variables.shp', decimal='.')
